In [42]:
import email.mime.multipart
import email.mime.nonmultipart
import email.policy
import json
import sys
import urllib.error
import urllib.request

import conllu
import pyterrier as pt

sys.path.insert(0, "..")

from src import udpipe, utils
from src.types import LAN, Experiment

In [2]:
if not pt.started():
    pt.init(tqdm="notebook", logging="INFO")

Picked up _JAVA_OPTIONS: -Djava.util.prefs.userRoot=/home/dburian/.config/java
PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.



## UDPipe


In [83]:


UDPipeService = "https://lindat.mff.cuni.cz/services/udpipe/api"


def perform_request(server, method, params={}):
    if not params:
        request_headers, request_data = {}, None
    else:
        message = email.mime.multipart.MIMEMultipart(
            "form-data", policy=email.policy.HTTP
        )

        for name, value in params.items():
            payload = email.mime.nonmultipart.MIMENonMultipart("text", "plain")
            payload.add_header(
                "Content-Disposition", 'form-data; name="{}"'.format(name)
            )
            payload.add_header("Content-Transfer-Encoding", "8bit")
            payload.set_payload(value, charset="utf-8")
            message.attach(payload)

        request_data = message.as_bytes().split(b"\r\n\r\n", maxsplit=1)[1]
        request_headers = {"Content-Type": message["Content-Type"]}

    try:
        with urllib.request.urlopen(
            urllib.request.Request(
                url="{}/{}".format(server, method),
                headers=request_headers,
                data=request_data,
            )
        ) as request:
            return json.loads(request.read())
    except urllib.error.HTTPError as e:
        print(
            "An exception was raised during UDPipe 'process' REST request.\n"
            "The service returned the following error:\n"
            "  {}".format(e.fp.read().decode("utf-8")),
            file=sys.stderr,
        )
        raise
    except json.JSONDecodeError as e:
        print(
            "Cannot parse the JSON response of UDPipe 'process' REST request.\n"
            "  {}".format(e.msg),
            file=sys.stderr,
        )
        raise


def process(args: dict[str, str], text_data):
    data = {
        "input": args["input"],
        "output": args["output"],
        "data": text_data,
    }
    for option in ["model", "tokenizer", "parser", "tagger"]:
        value = args.get(option, None)
        if value is not None:
            data[option] = value

    response = perform_request(UDPipeService, "process", data)
    if "model" not in response or "result" not in response:
        raise ValueError("Cannot parse the UDPipe 'process' REST request response.")

    print(
        "UDPipe generated an output using the model '{}'.".format(response["model"]),
        file=sys.stderr,
    )
    print(
        "Please respect the model licence (CC BY-NC-SA unless stated otherwise).",
        file=sys.stderr,
    )

    return response["result"]

In [127]:
args = {
    "input": "horizontal",
    "output": "conllu",
    "model": "czech",
    "tokenizer": "",
    "tagger": "",
}
text_data = """Ahoj jak se máš já se mám dobře

babička koupila rohlíky za 20 Kč"""

return_data = process(args, text_data)

print(return_data)

# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
1	Ahoj	ahoj	PART	TT-------------	_	_	_	_	_
2	jak	jak	ADV	Db-------------	PronType=Int,Rel	_	_	_	_
3	se	se	PRON	P7-X4----------	Case=Acc|PronType=Prs|Reflex=Yes|Variant=Short	_	_	_	_
4	máš	mít	VERB	VB-S---2P-AA---	Mood=Ind|Number=Sing|Person=2|Polarity=Pos|Tense=Pres|VerbForm=Fin|Voice=Act	_	_	_	_
5	já	já	PRON	PP-S1--1-------	Case=Nom|Number=Sing|Person=1|PronType=Prs	_	_	_	_
6	se	se	PRON	P7-X4----------	Case=Acc|PronType=Prs|Reflex=Yes|Variant=Short	_	_	_	_
7	mám	mít	VERB	VB-S---1P-AA---	Mood=Ind|Number=Sing|Person=1|Polarity=Pos|Tense=Pres|VerbForm=Fin|Voice=Act	_	_	_	_
8	dobře	dobře	ADV	Dg-------1A----	Degree=Pos|Polarity=Pos	_	_	_	_

# newpar
# sent_id = 2
1	babička	babička	NOUN	NNFS1-----A----	Case=Nom|Gender=Fem|Number=Sing|Polarity=Pos	_	_	_	_
2	koupila	koupit	VERB	VpQW---XR-AA---	Aspect=Perf|Gender=Fem,N

UDPipe generated an output using the model 'czech-pdt-ud-2.10-220711'.
Please respect the model licence (CC BY-NC-SA unless stated otherwise).


In [126]:

from conllu import parse

data = parse(return_data)

for sentence in data:
    for token in sentence:
        print(token["lemma"])

    print()
    print("new sentence")

ahoj
jak
se
mít
já
se
mít
dobře

new sentence
babička
koupit
rohlík
za
20
Kč

new sentence


In [32]:

ref_index = pt.IndexFactory.of("../indices/run-0_cs")
index = pt.IndexFactory.of("../indices/run-0-udpipe-lemm_cs")

08:44:46.459 [main] INFO org.terrier.structures.FSADocumentIndex - Document index requires 319.3 KiB remaining heap is 3.9 GiB
08:44:46.512 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file into memory
08:44:46.516 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading reverse map for key docno directly from disk
08:44:46.518 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta loading data file into memory
08:44:46.523 [main] INFO org.terrier.structures.FSADocumentIndex - Document index requires 319.3 KiB remaining heap is 3.8 GiB
08:44:46.558 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file into memory
08:44:46.563 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading reverse map for key docno directly from disk
08:44:46.564 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta loadin

In [33]:
print(index.getCollectionStatistics())
print(ref_index.getCollectionStatistics())

Number of documents: 81735
Number of terms: 243646
Number of postings: 11942065
Number of fields: 1
Number of tokens: 23922334
Field names: [text]
Positions:   false

Number of documents: 81735
Number of terms: 555598
Number of postings: 14248755
Number of fields: 1
Number of tokens: 23803359
Field names: [text]
Positions:   false



In [34]:
di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()
docid = 1  # docids are 0-based
# NB: postings will be null if the document is empty
for posting in di.getPostings(doi.getDocumentEntry(docid)):
    termid = posting.getId()
    lee = lex.getLexiconEntry(termid)
    print("%s with frequency %d" % (lee.getKey(), posting.getFrequency()))

01 with frequency 1
za with frequency 2
svůj with frequency 2
bankovka with frequency 1
zatím with frequency 1
o with frequency 1
02 with frequency 2
používat with frequency 1
se with frequency 1
unie with frequency 1
tento with frequency 1
okamžik with frequency 1
příchod with frequency 1
brusel with frequency 2
oslava with frequency 1
měna with frequency 2
být with frequency 1
euro with frequency 4
ohňostroj with frequency 2
mince with frequency 1
na with frequency 4
mít with frequency 2
evropa with frequency 1
v with frequency 3
s with frequency 3
k with frequency 2
a with frequency 7
který with frequency 1
jeden with frequency 1
evropský with frequency 5
až with frequency 1
připojit with frequency 1
metropole with frequency 1
evokovat with frequency 1
průhledný with frequency 1
pódium with frequency 1
most with frequency 1
organizátor with frequency 1
umístěný with frequency 1
bez with frequency 1
různorodost with frequency 1
vážný with frequency 1
smetana with frequency 1
několik 

In [35]:

document_paths = list(utils.paths_gen("../A1/documents_cs.lst", "../A1/documents_cs"))
documents = pt.index.treccollection2textgen(document_paths)

In [36]:
doc1 = next(documents)
doc2 = next(documents)

08:45:38.720 [main] INFO org.terrier.indexing.MultiDocumentFileCollection - TRECCollection 0% processing ../A1/documents_cs/ln020102.xml


In [37]:
print(doc1)
print("-----")
print(doc2)

{'docno': 'LN-20020102001', 'text': '01/02/02  304 miliony lidí ve 12 zemích si začalo vyměňovat své peníze za eura. Největší měnová revoluce v dějinách evropského kontinentu začala včera o půlnoci.  BRUSEL  Příchod nové evropské měny o půlnoci na 1. leden provázely v celé Evropě bouřlivé oslavy a ohňostroje. Lidé stáli fronty před bankomaty na eura, která si pak většinou dlouho prohlíželi. "Nevypadají jako opravdové. Jsou malé, nesmrdí jako peníze," hodnotil nové bankovky Kieran O\'Brien, turista v Amsterdamu. Přechod na euro probíhá zatím hladce a občané dvanácti zemí jsou většinou nadšeni a zvědaví na novou měnu, radovala se Evropská komise. "Nic zvláštního se nestalo. V tomto případě jsou žádné zprávy nejlepší zprávy," řekl její mluvčí Thomas Gerassimos. Pravou zkoušku má však nová měna teprve před sebou. Dnes se otevřou všechny banky a obchody. A teprve v tento okamžik se uvidí, zda se obchodníci dostatečně předzásobili eurem a vyškolili personál, aby s novou měnou uměl zacházet. 

In [38]:
print(udpipe.sanitize_text(doc1["text"]))
print("-----")
print(udpipe.sanitize_text(doc2["text"]))

01/02/02  304 miliony lidí ve 12 zemích si začalo vyměňovat své peníze za eura  Největší měnová revoluce v dějinách evropského kontinentu začala včera o půlnoci   BRUSEL  Příchod nové evropské měny o půlnoci na 1  leden provázely v celé Evropě bouřlivé oslavy a ohňostroje  Lidé stáli fronty před bankomaty na eura  která si pak většinou dlouho prohlíželi   Nevypadají jako opravdové  Jsou malé  nesmrdí jako peníze   hodnotil nové bankovky Kieran O Brien  turista v Amsterdamu  Přechod na euro probíhá zatím hladce a občané dvanácti zemí jsou většinou nadšeni a zvědaví na novou měnu  radovala se Evropská komise   Nic zvláštního se nestalo  V tomto případě jsou žádné zprávy nejlepší zprávy   řekl její mluvčí Thomas Gerassimos  Pravou zkoušku má však nová měna teprve před sebou  Dnes se otevřou všechny banky a obchody  A teprve v tento okamžik se uvidí  zda se obchodníci dostatečně předzásobili eurem a vyškolili personál  aby s novou měnou uměl zacházet  Evropská unie se snažila přechod přede

In [39]:

lemmatized_docs = udpipe.lemmatize_doc(
    doc1, lan=LAN.CS, udpipe_service=udpipe.UD_PIPE_SERVICE
)

# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = 01/02/02 304 miliony lidí ve 12 zemích si začalo vyměňovat své peníze za eura.
1	01	01	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpaceAfter=No
2	/	/	PUNCT	Z:-------------	_	_	_	_	SpaceAfter=No
3	02	02	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpaceAfter=No
4	/	/	PUNCT	Z:-------------	_	_	_	_	SpaceAfter=No
5	02	02	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpacesAfter=\s\s
6	304	304	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	_
7	miliony	milión	NOUN	NNIP1-----A----	Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur|Polarity=Pos	_	_	_	_
8	lidí	člověk	NOUN	NNMP2-----A----	Animacy=Anim|Case=Gen|Gender=Masc|Number=Plur|Polarity=Pos	_	_	_	_
9	ve	v	ADP	RV--6----------	AdpType=Voc|Case=Loc	_	_	_	_
10	12	12	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	_
11	zemích	země

In [44]:
queries = utils.read_topics(
    "../A1/topics-train_cs.xml",
    Experiment(index_dir="adfas", lan=LAN.CS).get_query_parser(),
)

print(queries)

               qid                                              query
0   10.2452/401-AH                                       Inflace Eura
1   10.2452/402-AH                                 Obnovitelné zdroje
2   10.2452/403-AH                                     Role policisty
3   10.2452/404-AH                Summit NATO a bezpečnostní opatření
4   10.2452/405-AH                                    Astma u dětství
5   10.2452/406-AH                                    Animované filmy
6   10.2452/407-AH                                 Australský premiér
7   10.2452/408-AH                                     Klonování lidí
8   10.2452/409-AH                 Automobilové bombové útoky na Bali
9   10.2452/410-AH  Porušení programu jaderných zbraní Severní Koreou
10  10.2452/411-AH                             Oskar za nejlepší film
11  10.2452/412-AH                                 Knihy o politicích
12  10.2452/413-AH              Snižování rizika onemocnění cukrovkou
13  10.2452/414-AH  

In [45]:
lemma_queries = udpipe.lemmatize_topics(
    queries, lan=LAN.CS, udpipe_service=udpipe.UD_PIPE_SERVICE
)

# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = Inflace Eura
1	Inflace	inflace	NOUN	NNFS1-----A----	Case=Nom|Gender=Fem|Number=Sing|Polarity=Pos	_	_	_	_
2	Eura	Eura	PROPN	NNFS1-----A----	Case=Nom|Gender=Fem|NameType=Com|Number=Sing|Polarity=Pos	_	_	_	SpaceAfter=No


# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = Obnovitelné zdroje
1	Obnovitelné	obnovitelný	ADJ	AAIP1----1A----	Animacy=Inan|Case=Nom|Degree=Pos|Gender=Masc|Number=Plur|Polarity=Pos	_	_	_	_
2	zdroje	zdroj	NOUN	NNIP1-----A----	Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur|Polarity=Pos	_	_	_	SpaceAfter=No


# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
#

# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = Únosy letadel
1	Únosy	únos	NOUN	NNIP1-----A----	Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur|Polarity=Pos	_	_	_	_
2	letadel	letadlo	NOUN	NNNP2-----A----	Case=Gen|Gender=Neut|Number=Plur|Polarity=Pos	_	_	_	SpaceAfter=No


# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = Prohlášení Bülenta Ecevita
1	Prohlášení	prohlášení	NOUN	NNNS1-----A----	Case=Nom|Gender=Neut|Number=Sing|Polarity=Pos|VerbForm=Vnoun	_	_	_	_
2	Bülenta	Bülent	PROPN	NNMS2-----A----	Animacy=Anim|Case=Gen|Gender=Masc|NameType=Sur|Number=Sing|Polarity=Pos	_	_	_	_
3	Ecevita	Ecevit	PROPN	NNFS1-----A----	Animacy=Anim|Case=Gen|Gender=Masc|NameType=Sur|Number=Sing|Polarity=Pos	_	_	_	SpaceAfter=No


# generator

In [46]:
print(lemma_queries)

               qid                                         query
0   10.2452/401-AH                                  inflace Eura
1   10.2452/402-AH                             obnovitelný zdroj
2   10.2452/403-AH                                role policista
3   10.2452/404-AH           summit NATO a bezpečnostní opatření
4   10.2452/405-AH                               astma u dětství
5   10.2452/406-AH                                animovaný film
6   10.2452/407-AH                            australský premiér
7   10.2452/408-AH                              klonování člověk
8   10.2452/409-AH             automobilový bombový útok na Bali
9   10.2452/410-AH  porušení program jaderný zbraň severní Korea
10  10.2452/411-AH                           Oskar za dobrý film
11  10.2452/412-AH                              kniha o politice
12  10.2452/413-AH          snižování riziko onemocnění cukrovka
13  10.2452/414-AH                                pivní festival
14  10.2452/415-AH       